In [5]:
"""
This script processes all images in a folder, detects the largest square-like marker,
applies a perspective transform to extract it cleanly, and saves the result.

Author: [Juhee Hur]
Date: 2025-07-24
"""

import cv2
import numpy as np
import os
import glob


def extract_robust_square_marker(img: np.ndarray) -> np.ndarray | None:
    """
    Detects the largest square marker in the image and extracts it with a perspective transform.

    Args:
        img (np.ndarray): Input image (BGR).

    Returns:
        np.ndarray | None: Extracted square region as a new image. None if not found.
    """
    # Step 1: Convert to grayscale and apply Otsu thresholding
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bin_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Step 2: Find external contours on the inverted binary image
    contours, _ = cv2.findContours(255 - bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    candidates = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 5000:  # Filter out small contours
            continue

        epsilon = 0.03 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # Check for convex quadrilaterals (approx. square)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            rect = approx.reshape(4, 2)
            width = np.linalg.norm(rect[0] - rect[1])
            height = np.linalg.norm(rect[1] - rect[2])
            ratio = width / height if height != 0 else 0

            if abs(1 - ratio) < 0.3:  # Allow up to 30% aspect ratio distortion
                candidates.append((area, rect))

    if not candidates:
        return None

    # Step 3: Pick the largest square candidate
    _, best_rect = max(candidates, key=lambda x: x[0])

    # Step 4: Order corners (top-left, top-right, bottom-right, bottom-left)
    def order_points(pts: np.ndarray) -> np.ndarray:
        s = pts.sum(axis=1)
        diff = np.diff(pts, axis=1)

        rect = np.zeros((4, 2), dtype='float32')
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]
        return rect

    rect = order_points(best_rect)

    # Step 5: Apply perspective transform to obtain a top-down view
    size = int(max(
        np.linalg.norm(rect[0] - rect[1]),
        np.linalg.norm(rect[1] - rect[2]),
        np.linalg.norm(rect[2] - rect[3]),
        np.linalg.norm(rect[3] - rect[0])
    ))

    dst_pts = np.array([
        [0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]
    ], dtype='float32')

    M = cv2.getPerspectiveTransform(rect, dst_pts)
    warped = cv2.warpPerspective(img, M, (size, size))

    return warped


def process_folder(input_dir: str = "./images", output_dir: str = "./output") -> None:
    """
    Process all images in a folder, extract square markers, and save results.

    Args:
        input_dir (str): Folder path containing input images.
        output_dir (str): Folder path to save extracted images.
    """
    os.makedirs(output_dir, exist_ok=True)
    image_paths = glob.glob(os.path.join(input_dir, "*.*"))

    for path in image_paths:
        filename = os.path.basename(path)
        img = cv2.imread(path)

        if img is None:
            print(f"❌ Failed to load image: {filename}")
            continue

        result = extract_robust_square_marker(img)
        if result is not None:
            save_path = os.path.join(output_dir, f"extracted_{filename}")
            cv2.imwrite(save_path, result)
            print(f"✅ Extracted: {filename}")
        else:
            print(f"⚠️ No square marker found: {filename}")


if __name__ == "__main__":
    process_folder()


✅ Extracted: left_2.png
✅ Extracted: left_3.png
✅ Extracted: left_1.png
✅ Extracted: left_4.png
✅ Extracted: right_1.png
✅ Extracted: right_3.png
✅ Extracted: right_2.png
✅ Extracted: right_4.png


In [4]:
import cv2
import numpy as np
import os
import glob
import json

def order_points(pts: np.ndarray) -> np.ndarray:
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect = np.zeros((4, 2), dtype='float32')
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def extract_robust_square_marker(img: np.ndarray) -> np.ndarray | None:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bin_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(255 - bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 5_000:
            continue
        eps = 0.03 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps, True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            pts = approx.reshape(4,2)
            w = np.linalg.norm(pts[0] - pts[1])
            h = np.linalg.norm(pts[1] - pts[2])
            if abs(1 - w/h) < 0.3:
                candidates.append((area, pts))
    if not candidates:
        return None
    _, best = max(candidates, key=lambda x: x[0])
    rect = order_points(best)
    size = int(max(
        np.linalg.norm(rect[0]-rect[1]),
        np.linalg.norm(rect[1]-rect[2]),
        np.linalg.norm(rect[2]-rect[3]),
        np.linalg.norm(rect[3]-rect[0])
    ))
    dst = np.array([[0,0],[size-1,0],[size-1,size-1],[0,size-1]], dtype='float32')
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(img, M, (size, size))

def mark_and_offset(image: np.ndarray):
    h, w = image.shape[:2]
    cx0, cy0 = w/2, h/2

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ranges = {
        'C': ((90,80,80),(130,255,255)),
        'M': ((130,50,70),(170,255,255)),
        'Y': ((20,80,80),(40,255,255)),
        'K': ((0,0,0),(180,255,70)),
    }

    centers = {}
    offsets = {}

    for col, (lo, hi) in ranges.items():
        mask = cv2.inRange(hsv, np.array(lo), np.array(hi))
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not cnts:
            continue
        big = max(cnts, key=cv2.contourArea)
        M = cv2.moments(big)
        if M["m00"] == 0:
            continue
        cx = int(M["m10"]/M["m00"])
        cy = int(M["m01"]/M["m00"])
        centers[col] = (cx, cy)

        # compute signed offsets
        if col == 'K':
            dx = cx0 - cx;    dy = cy0 - cy
        elif col == 'C':
            dx = cx - cx0;    dy = cy0 - cy
        elif col == 'M':
            dx = cx0 - cx;    dy = cy - cy0
        else:  # 'Y'
            dx = cx - cx0;    dy = cy - cy0

        offsets[col] = (int(dx), int(dy))

        # draw
        cv2.circle(image, (cx, cy), 10, (0,255,0), -1)
        cv2.putText(image, f"{col}{offsets[col]}", (cx+5, cy-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

    # compare to K
    if 'K' in offsets:
        kdx, kdy = offsets['K']
        comp = {}
        for col in ('C','M','Y'):
            if col in offsets:
                cdx,cdy = offsets[col]
                comp[col] = (cdx - kdx, cdy - kdy)
        offsets['delta_to_K'] = comp

    return image, centers, offsets

def process_all(input_dir: str = "./output", out_dir: str = "./analysis"):
    os.makedirs(out_dir, exist_ok=True)
    summary = {}
    for path in glob.glob(os.path.join(input_dir, "extracted_*.*")):
        img = cv2.imread(path)
        if img is None:
            continue
        marked, ctrs, offs = mark_and_offset(img)
        name = os.path.basename(path)
        cv2.imwrite(os.path.join(out_dir, f"marked_{name}"), marked)
        summary[name] = {"centers": ctrs, "offsets": offs}

    # dump JSON
    with open(os.path.join(out_dir, "cmyk_summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

if __name__ == "__main__":
    process_all()


In [11]:
import cv2
import numpy as np
import glob
import os

def compute_y_minus_x(image: np.ndarray,
                      hsv_ranges: dict[str, tuple[tuple[int,int,int], tuple[int,int,int]]],
                      min_area_ratio: float = 0.5
                     ) -> dict[str, tuple[tuple[int,int], tuple[int,int], int]]:
    """
    Detects the bottom-left and bottom-right corners of each CMY block,
    then computes y_minus_x = left_y - right_y for each color.

    Args:
        image: BGR image of the extracted marker.
        hsv_ranges: Dict mapping 'C','M','Y' to (lower_HSV, upper_HSV).
        min_area_ratio: Minimum contour area relative to the max area to keep.

    Returns:
        A dict mapping each color to (left_pt, right_pt, y_minus_x).
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    results: dict[str, tuple[tuple[int,int], tuple[int,int], int]] = {}

    def detect_quad(c: np.ndarray) -> np.ndarray | None:
        """Approximate a convex quadrilateral from the convex hull of contour c."""
        hull = cv2.convexHull(c)
        eps = 0.02 * cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1, 2)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            return approx
        return None

    def detect_block(color: str):
        """
        For a given color, apply morphology, hull, and quad detection.
        Returns (left_pt, right_pt) or None if detection fails.
        """
        lower, upper = hsv_ranges[color]
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))

        # Remove thin lines or speckles with morphological closing & opening
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            print(f"⚠️ No {color} contours found")
            return None

        # Filter out small contours
        areas = [cv2.contourArea(c) for c in contours]
        max_area = max(areas)
        large_contours = [c for c, a in zip(contours, areas) if a >= max_area * min_area_ratio]
        if not large_contours:
            print(f"⚠️ No large {color} contours found")
            return None

        # Try each large contour for a quad
        for cnt in sorted(large_contours, key=cv2.contourArea, reverse=True):
            quad = detect_quad(cnt)
            if quad is not None:
                # Select the two bottom points by largest y
                bottom_pts = sorted(quad, key=lambda p: p[1], reverse=True)[:2]
                left_pt, right_pt = tuple(sorted(bottom_pts, key=lambda p: p[0])[0]), \
                                    tuple(sorted(bottom_pts, key=lambda p: p[0])[1])
                return left_pt, right_pt

        print(f"⚠️ {color} quadrilateral not detected after hull")
        return None

    # Process each CMY color
    for color in ('C', 'M', 'Y'):
        pts = detect_block(color)
        if pts:
            left_pt, right_pt = pts
            y_minus_x = int(left_pt[1] - right_pt[1])
            results[color] = (left_pt, right_pt, y_minus_x)

    return results


if __name__ == "__main__":
    # Define HSV ranges for Cyan, Magenta, Yellow
    HSV_RANGES = {
        'C': ((90, 80, 80), (130, 255, 255)),
        'M': ((130, 50, 70), (170, 255, 255)),
        'Y': ((20, 80, 80), (40, 255, 255)),
    }

    os.makedirs('./cocking_debug', exist_ok=True)

    # Iterate over every extracted marker image
    for img_path in glob.glob('./output/extracted_*.png'):
        image = cv2.imread(img_path)
        if image is None:
            continue

        results = compute_y_minus_x(image, HSV_RANGES)

        # Draw and annotate debug markers
        for color, (left_pt, right_pt, delta) in results.items():
            cv2.circle(image, left_pt, 6, (0, 255, 0), -1)   # left corner in green
            cv2.circle(image, right_pt, 6, (255, 0, 0), -1)  # right corner in blue
            cv2.putText(image, f"{color}: {delta}px",
                        (left_pt[0], left_pt[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Print numeric results to console
        print(os.path.basename(img_path), {c: results[c][2] for c in results})

        # Save the annotated debug image
        debug_path = os.path.join('./cocking_debug', 'dbg_' + os.path.basename(img_path))
        cv2.imwrite(debug_path, image)


extracted_left_3.png {'C': 2, 'M': 1, 'Y': -2}
extracted_left_2.png {'C': -1, 'M': 0, 'Y': -2}
extracted_left_1.png {'C': -2, 'M': -2, 'Y': -2}
extracted_left_4.png {'C': 3, 'M': 2, 'Y': 1}
extracted_20250718_145610755.png {'C': -2, 'M': -2, 'Y': -2}
extracted_20250718_145635599.png {'C': 3, 'M': 2, 'Y': 1}
extracted_20250718_145634047.png {'C': -1, 'M': 0, 'Y': -2}
extracted_20250718_145634895.png {'C': 2, 'M': 1, 'Y': -2}
extracted_right_1.png {'C': 1, 'M': 1, 'Y': -1}
extracted_right_3.png {'C': 2, 'M': 1, 'Y': -2}
extracted_right_2.png {'C': 1, 'M': 2, 'Y': -1}
extracted_right_4.png {'C': -1, 'M': 3, 'Y': -3}


In [12]:
import cv2
import numpy as np
import glob
import os

def detect_and_mark_black_border(image: np.ndarray) -> np.ndarray:
    """
    Detects the largest square (black border) in the image, even if
    parts of it are overlapped by other colors, and draws its contour.
    Returns a copy of the image with the border marked.
    """
    vis = image.copy()
    # 1. Convert to grayscale and threshold for dark pixels (the black border)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV)
    
    # 2. Morphological closing to fill small gaps, then opening to remove noise
    kern = cv2.getStructuringElement(cv2.MORPH_RECT, (25,25))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kern)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kern)
    
    # 3. Find external contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return vis  # nothing found
    
    # 4. Filter contours for convex quadrilaterals
    quads = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 10000:  # skip small noise
            continue
        eps = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps, True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            quads.append((area, approx.reshape(4,2)))
    
    if not quads:
        return vis
    
    # 5. Pick the largest quad
    _, best = max(quads, key=lambda x: x[0])
    
    # 6. Draw it
    cv2.polylines(vis, [best], True, (0,0,255), thickness=4)  # red border
    # Optionally draw corner circles:
    for (x,y) in best:
        cv2.circle(vis, (x,y), 8, (0,255,0), -1)
    
    return vis

if __name__ == "__main__":
    os.makedirs("./border_debug", exist_ok=True)
    for path in glob.glob("./output/extracted_*.png"):
        img = cv2.imread(path)
        if img is None:
            continue

        marked = detect_and_mark_black_border(img)
        out_path = path.replace("./output/", "./border_debug/dbg_")
        cv2.imwrite(out_path, marked)
        print(f"Processed {os.path.basename(path)} -> {os.path.basename(out_path)}")


Processed extracted_left_3.png -> dbg_extracted_left_3.png
Processed extracted_left_2.png -> dbg_extracted_left_2.png
Processed extracted_left_1.png -> dbg_extracted_left_1.png
Processed extracted_left_4.png -> dbg_extracted_left_4.png
Processed extracted_20250718_145610755.png -> dbg_extracted_20250718_145610755.png
Processed extracted_20250718_145635599.png -> dbg_extracted_20250718_145635599.png
Processed extracted_20250718_145634047.png -> dbg_extracted_20250718_145634047.png
Processed extracted_20250718_145634895.png -> dbg_extracted_20250718_145634895.png
Processed extracted_right_1.png -> dbg_extracted_right_1.png
Processed extracted_right_3.png -> dbg_extracted_right_3.png
Processed extracted_right_2.png -> dbg_extracted_right_2.png
Processed extracted_right_4.png -> dbg_extracted_right_4.png


In [13]:
import cv2
import numpy as np
import glob
import os

def compute_y_minus_x_px(image: np.ndarray,
                         hsv_ranges: dict[str, tuple[tuple[int,int,int], tuple[int,int,int]]],
                         min_area_ratio: float = 0.5
                        ) -> dict[str, tuple[tuple[int,int], tuple[int,int], int]]:
    """
    Same as before: returns y_minus_x in pixels for each CMY block.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    results: dict[str, tuple[tuple[int,int], tuple[int,int], int]] = {}

    def detect_quad(c: np.ndarray) -> np.ndarray | None:
        hull = cv2.convexHull(c)
        eps = 0.02 * cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1, 2)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            return approx
        return None

    def detect_block(color: str):
        lower, upper = hsv_ranges[color]
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not cnts:
            return None
        areas = [cv2.contourArea(c) for c in cnts]
        max_area = max(areas)
        big = [c for c,a in zip(cnts, areas) if a >= max_area * min_area_ratio]
        for cnt in sorted(big, key=cv2.contourArea, reverse=True):
            quad = detect_quad(cnt)
            if quad is not None:
                bottom = sorted(quad, key=lambda p: p[1], reverse=True)[:2]
                left_pt, right_pt = tuple(sorted(bottom, key=lambda p: p[0])[0]), \
                                    tuple(sorted(bottom, key=lambda p: p[0])[1])
                y_minus_x = int(left_pt[1] - right_pt[1])
                return left_pt, right_pt, y_minus_x
        return None

    for color in ('C', 'M', 'Y'):
        res = detect_block(color)
        if res:
            results[color] = res

    return results

if __name__ == "__main__":
    HSV_RANGES = {
        'C': ((90, 80, 80), (130, 255, 255)),
        'M': ((130, 50, 70), (170, 255, 255)),
        'Y': ((20, 80, 80), (40, 255, 255)),
    }

    os.makedirs('./cocking_debug', exist_ok=True)

    for img_path in glob.glob('./output/extracted_*.png'):
        image = cv2.imread(img_path)
        if image is None:
            continue

        # 1) Compute px deltas
        px_results = compute_y_minus_x_px(image, HSV_RANGES)

        # 2) Compute scale: 5 mm real height / image_height_px
        height_px = image.shape[0]
        mm_per_px = 5.0 / height_px

        # 3) Convert each delta to mm (float, e.g. one decimal)
        mm_results: dict[str, float] = {}
        for color, (_, _, delta_px) in px_results.items():
            delta_mm = delta_px * mm_per_px
            mm_results[color] = round(delta_mm, 2)

        # 4) Annotate and save debug image
        for color, (left_pt, right_pt, delta_px) in px_results.items():
            delta_mm = mm_results[color]
            cv2.circle(image, left_pt, 6, (0, 255, 0), -1)
            cv2.circle(image, right_pt, 6, (255, 0, 0), -1)
            cv2.putText(image,
                        f"{color}: {delta_px}px / {delta_mm}mm",
                        (left_pt[0], left_pt[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # 5) Print mm results to console
        print(os.path.basename(img_path), mm_results)

        # 6) Save annotated debug image
        debug_path = os.path.join('./cocking_debug', 'dbg_' + os.path.basename(img_path))
        cv2.imwrite(debug_path, image)


extracted_left_3.png {'C': 0.02, 'M': 0.01, 'Y': -0.02}
extracted_left_2.png {'C': -0.01, 'M': 0.0, 'Y': -0.02}
extracted_left_1.png {'C': -0.02, 'M': -0.02, 'Y': -0.02}
extracted_left_4.png {'C': 0.02, 'M': 0.02, 'Y': 0.01}
extracted_20250718_145610755.png {'C': -0.02, 'M': -0.02, 'Y': -0.02}
extracted_20250718_145635599.png {'C': 0.02, 'M': 0.02, 'Y': 0.01}
extracted_20250718_145634047.png {'C': -0.01, 'M': 0.0, 'Y': -0.02}
extracted_20250718_145634895.png {'C': 0.02, 'M': 0.01, 'Y': -0.02}
extracted_right_1.png {'C': 0.01, 'M': 0.01, 'Y': -0.01}
extracted_right_3.png {'C': 0.02, 'M': 0.01, 'Y': -0.02}
extracted_right_2.png {'C': 0.01, 'M': 0.02, 'Y': -0.01}
extracted_right_4.png {'C': -0.01, 'M': 0.02, 'Y': -0.02}


In [15]:
import cv2
import numpy as np
import glob
import os
import math

def compute_y_minus_x_px(image: np.ndarray,
                         hsv_ranges: dict[str, tuple[tuple[int,int,int], tuple[int,int,int]]],
                         min_area_ratio: float = 0.5
                        ) -> dict[str, tuple[tuple[int,int], tuple[int,int], int]]:
    """
    Detects the bottom-left and bottom-right corners of each CMY block,
    then computes y_minus_x = left_y - right_y in pixels.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    results: dict[str, tuple[tuple[int,int], tuple[int,int], int]] = {}

    def detect_quad(c: np.ndarray) -> np.ndarray | None:
        hull = cv2.convexHull(c)
        eps = 0.02 * cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1, 2)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            return approx
        return None

    def detect_block(color: str):
        lower, upper = hsv_ranges[color]
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None
        areas = [cv2.contourArea(c) for c in contours]
        max_area = max(areas)
        big = [c for c,a in zip(contours, areas) if a >= max_area * min_area_ratio]
        for cnt in sorted(big, key=cv2.contourArea, reverse=True):
            quad = detect_quad(cnt)
            if quad is not None:
                bottom = sorted(quad, key=lambda p: p[1], reverse=True)[:2]
                left_pt, right_pt = tuple(sorted(bottom, key=lambda p: p[0])[0]), \
                                    tuple(sorted(bottom, key=lambda p: p[0])[1])
                y_minus_x = int(left_pt[1] - right_pt[1])
                return left_pt, right_pt, y_minus_x
        return None

    for color in ('C', 'M', 'Y'):
        res = detect_block(color)
        if res:
            results[color] = res

    return results


if __name__ == "__main__":
    # 1) Read user input for marker width in mm
    width_mm = float(input("Enter the marker WIDTH in mm: "))

    # 2) HSV ranges for CMY
    HSV_RANGES = {
        'C': ((90, 80, 80), (130, 255, 255)),
        'M': ((130, 50, 70), (170, 255, 255)),
        'Y': ((20, 80, 80), (40, 255, 255)),
    }

    os.makedirs('./cocking_debug', exist_ok=True)

    # 3) Process each extracted image
    for img_path in glob.glob('./output/extracted_*.png'):
        image = cv2.imread(img_path)
        if image is None:
            continue

        # a) get pixel deltas
        px_results = compute_y_minus_x_px(image, HSV_RANGES)

        # b) compute mm-per-px
        height_px = image.shape[0]
        mm_per_px = 5.0 / height_px

        # c) compute tilt for each color
        tilt_results = {}
        for color, (_, _, delta_px) in px_results.items():
            delta_mm = delta_px * mm_per_px
            denom = math.sqrt(1.5**2 - delta_mm**2)
            tilt = width_mm * (delta_mm / denom) if denom != 0 else float('nan')
            tilt_results[color] = round(tilt, 3)

        # d) annotate and save
        for color, (lp, rp, delta_px) in px_results.items():
            tilt = tilt_results[color]
            text = f"{color}: tilt={tilt}mm"
            cv2.circle(image, lp, 6, (0,255,0), -1)
            cv2.circle(image, rp, 6, (255,0,0), -1)
            cv2.putText(image, text, (lp[0], lp[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

        print(os.path.basename(img_path), tilt_results)
        debug_path = os.path.join('./cocking_debug', 'dbg_' + os.path.basename(img_path))
        cv2.imwrite(debug_path, image)


Enter the marker WIDTH in mm:  510


extracted_left_3.png {'C': 5.397, 'M': 2.698, 'Y': -5.397}
extracted_left_2.png {'C': -2.69, 'M': 0.0, 'Y': -5.38}
extracted_left_1.png {'C': -5.406, 'M': -5.406, 'Y': -5.406}
extracted_left_4.png {'C': 8.109, 'M': 5.406, 'Y': 2.703}
extracted_20250718_145610755.png {'C': -5.406, 'M': -5.406, 'Y': -5.406}
extracted_20250718_145635599.png {'C': 8.109, 'M': 5.406, 'Y': 2.703}
extracted_20250718_145634047.png {'C': -2.69, 'M': 0.0, 'Y': -5.38}
extracted_20250718_145634895.png {'C': 5.397, 'M': 2.698, 'Y': -5.397}
extracted_right_1.png {'C': 2.703, 'M': 2.703, 'Y': -2.703}
extracted_right_3.png {'C': 5.406, 'M': 2.703, 'Y': -5.406}
extracted_right_2.png {'C': 2.698, 'M': 5.397, 'Y': -2.698}
extracted_right_4.png {'C': -2.716, 'M': 8.148, 'Y': -8.148}


In [19]:
import cv2
import numpy as np
import glob
import os

def order_points(pts: np.ndarray) -> np.ndarray:
    """Order 4 points as TL, TR, BR, BL."""
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect = np.zeros((4,2), dtype="float32")
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def extract_marker(image: np.ndarray) -> np.ndarray | None:
    """Crop out the largest near-square marker via perspective transform."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, bin_ = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts, _ = cv2.findContours(255 - bin_, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in cnts:
        a = cv2.contourArea(c)
        if a < 5000: continue
        eps = 0.03 * cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, eps, True)
        if len(approx)==4 and cv2.isContourConvex(approx):
            pts = approx.reshape(4,2)
            w = np.linalg.norm(pts[0]-pts[1])
            h = np.linalg.norm(pts[1]-pts[2])
            if abs(1 - w/h) < 0.3:
                candidates.append((a, pts))
    if not candidates:
        return None
    _, best = max(candidates, key=lambda x:x[0])
    rect = order_points(best)
    size = int(max(
        np.linalg.norm(rect[0]-rect[1]),
        np.linalg.norm(rect[1]-rect[2]),
        np.linalg.norm(rect[2]-rect[3]),
        np.linalg.norm(rect[3]-rect[0])
    ))
    dst = np.array([[0,0],[size-1,0],[size-1,size-1],[0,size-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (size,size))

def detect_bottom_left(img: np.ndarray, hsv_range: tuple, min_area_ratio=0.5):
    """
    Find the bottom‑left corner of the largest quadrilateral of given color.
    Returns (x_px, y_px) or None.
    """
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array(hsv_range[0]), np.array(hsv_range[1]))
    # clean up thin lines
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)

    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None

    areas = [cv2.contourArea(c) for c in cnts]
    max_a = max(areas)
    big = [c for c,a in zip(cnts,areas) if a >= max_a*min_area_ratio]
    for c in sorted(big, key=cv2.contourArea, reverse=True):
        hull = cv2.convexHull(c)
        eps = 0.02*cv2.arcLength(hull,True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1,2)
        if len(approx)==4 and cv2.isContourConvex(approx):
            rect = order_points(approx)
            # bottom two pts
            bottom = sorted(rect, key=lambda p:p[1], reverse=True)[:2]
            bl = tuple(sorted(bottom, key=lambda p:p[0])[0])
            return bl
    return None

if __name__ == "__main__":
    # HSV ranges for Cyan, Magenta, Yellow
    HSV = {
        'C': ((90,80,80),(130,255,255)),
        'M': ((130,50,70),(170,255,255)),
        'Y': ((20,80,80),(40,255,255)),
    }

    # reference lines per color
    # 'lat_dir': 'left' or 'right'
    # 'cir_dir': 'top' or 'bottom'
    REFS = {
        'C': {'lat':'right','cir':'top'},
        'M': {'lat':'left', 'cir':'bottom'},
        'Y': {'lat':'right','cir':'bottom'},
    }

    os.makedirs('./measurement', exist_ok=True)

    for path in glob.glob('./output/extracted_*.png'):
        img = cv2.imread(path)
        if img is None:
            continue

        cropped = extract_marker(img)
        if cropped is None:
            print(f"Marker not found in {path}")
            continue

        h_px, w_px = cropped.shape[:2]
        mm_per_px_x = 5.0 / w_px
        mm_per_px_y = 5.0 / h_px

        results = {}
        for col, hr in HSV.items():
            bl = detect_bottom_left(cropped, hr)
            if bl is None:
                results[col] = None
                continue

            x_px, y_px = bl

            # lateral px → mm
            if REFS[col]['lat']=='left':
                lat_px = x_px
            else:  # right
                lat_px = w_px - x_px
            lat_mm = lat_px * mm_per_px_x

            # circum px → mm
            if REFS[col]['cir']=='bottom':
                cir_px = y_px
            else:  # top
                cir_px = h_px - y_px
            cir_mm = cir_px * mm_per_px_y

            results[col] = {
                'lateral_mm': round(lat_mm, 3),
                'circum_mm':  round(cir_mm, 3)
            }

        # print per-image results
        print(os.path.basename(path), results)

        # optionally: save JSON per image
        with open(f"./measurement/{os.path.basename(path)}.json", "w") as f:
            json.dump(results, f, indent=2)


extracted_left_3.png {'C': {'lateral_mm': 2.329, 'circum_mm': 2.822}, 'M': {'lateral_mm': 0.89, 'circum_mm': 4.754}, 'Y': {'lateral_mm': 1.908, 'circum_mm': 4.658}}
extracted_left_2.png {'C': {'lateral_mm': 2.314, 'circum_mm': 2.813}, 'M': {'lateral_mm': 0.88, 'circum_mm': 4.746}, 'Y': {'lateral_mm': 1.902, 'circum_mm': 4.651}}
extracted_left_1.png {'C': {'lateral_mm': 2.333, 'circum_mm': 2.826}, 'M': {'lateral_mm': 0.868, 'circum_mm': 4.745}, 'Y': {'lateral_mm': 1.911, 'circum_mm': 4.666}}
extracted_left_4.png {'C': {'lateral_mm': 2.309, 'circum_mm': 2.811}, 'M': {'lateral_mm': 0.9, 'circum_mm': 4.769}, 'Y': {'lateral_mm': 1.887, 'circum_mm': 4.658}}
extracted_20250718_145610755.png {'C': {'lateral_mm': 2.333, 'circum_mm': 2.826}, 'M': {'lateral_mm': 0.868, 'circum_mm': 4.745}, 'Y': {'lateral_mm': 1.911, 'circum_mm': 4.666}}
extracted_20250718_145635599.png {'C': {'lateral_mm': 2.309, 'circum_mm': 2.811}, 'M': {'lateral_mm': 0.9, 'circum_mm': 4.769}, 'Y': {'lateral_mm': 1.887, 'circum

In [29]:
import cv2
import numpy as np
import glob
import os
import json

def order_points(pts: np.ndarray) -> np.ndarray:
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect = np.zeros((4,2), dtype="float32")
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def extract_marker(image: np.ndarray) -> np.ndarray | None:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, bin_ = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cnts, _ = cv2.findContours(255 - bin_, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cands = []
    for c in cnts:
        a = cv2.contourArea(c)
        if a < 5000: continue
        eps = 0.03 * cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, eps, True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            pts = approx.reshape(4,2)
            w = np.linalg.norm(pts[0] - pts[1])
            h = np.linalg.norm(pts[1] - pts[2])
            if abs(1 - w/h) < 0.3:
                cands.append((a, pts))
    if not cands:
        return None
    _, best = max(cands, key=lambda x: x[0])
    rect = order_points(best)
    size = int(max(
        np.linalg.norm(rect[0]-rect[1]),
        np.linalg.norm(rect[1]-rect[2]),
        np.linalg.norm(rect[2]-rect[3]),
        np.linalg.norm(rect[3]-rect[0])
    ))
    dst = np.array([[0,0],[size-1,0],[size-1,size-1],[0,size-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (size, size))

def detect_bottom_left(img: np.ndarray, hsv_range: tuple, min_area_ratio=0.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array(hsv_range[0]), np.array(hsv_range[1]))
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    areas = [cv2.contourArea(c) for c in cnts]
    mx = max(areas)
    big = [c for c,a in zip(cnts,areas) if a>=mx*min_area_ratio]
    for c in sorted(big, key=cv2.contourArea, reverse=True):
        hull = cv2.convexHull(c)
        eps = 0.02 * cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1,2)
        if len(approx)==4 and cv2.isContourConvex(approx):
            rect = order_points(approx)
            bottom = sorted(rect, key=lambda p:p[1], reverse=True)[:2]
            bl = tuple(sorted(bottom, key=lambda p:p[0])[0])
            return bl
    return None

if __name__ == "__main__":
    # HSV color ranges
    HSV = {
        'C': ((90,80,80),(130,255,255)),
        'M': ((130,50,70),(170,255,255)),
        'Y': ((20,80,80),(40,255,255)),
    }
    # reference directions
    INFO = {
        'C': {'lat_dir':'right',  'cir_dir':'top'},
        'M': {'lat_dir':'left',   'cir_dir':'bottom'},
        'Y': {'lat_dir':'right',  'cir_dir':'bottom'},
    }

    os.makedirs('./measurement', exist_ok=True)
    os.makedirs('./measurement/debug', exist_ok=True)

    for path in glob.glob('./output/extracted_*.png'):
        orig = cv2.imread(path)
        cropped = extract_marker(orig)
        if cropped is None:
            print(f"❌ Marker not found in {path}")
            continue

        h_px, w_px = cropped.shape[:2]
        mm_per_x = 5.0 / w_px
        mm_per_y = 5.0 / h_px

        results = {}
        debug = cropped.copy()

        cx, cy = w_px // 2, h_px // 2
        cv2.circle(debug, (cx, cy), 6, (0,0,255), -1)

        for col, hr in HSV.items():
            bl = detect_bottom_left(cropped, hr)
            if bl is None:
                results[col] = None
                continue

            x_px, y_px = bl
            bl_i = (int(x_px), int(y_px))
            info = INFO[col]

            # LATERAL
            lat_px = x_px if info['lat_dir'] == 'left' else w_px - x_px
            lat_mm = lat_px * mm_per_x

            # CIRCUMFERENTIAL
            cir_px = y_px if info['cir_dir'] == 'top' else h_px - y_px
            cir_mm = cir_px * mm_per_y

            # DELTA calculation per color
            if col == 'M':
                d_lat = 0.5 - lat_mm
                d_cir = 0.5 - cir_mm
            elif col == 'C':
                d_lat = lat_mm - 2.0
                d_cir = cir_mm - 2.0
            else:  # 'Y'
                d_lat = lat_mm - 2.0
                d_cir = 0.5 - cir_mm

            results[col] = {
                'measured_mm': (round(lat_mm,3), round(cir_mm,3)),
                'delta_mm':    (round(d_lat,3),  round(d_cir,3))
            }

            # debug visuals
            cv2.circle(debug, bl_i, 6, (0,255,0), -1)
            start_x = 0 if info['lat_dir']=='left' else w_px
            cv2.line(debug, (start_x, bl_i[1]), bl_i, (255,0,0), 2)
            start_y = 0 if info['cir_dir']=='top' else h_px
            cv2.line(debug, (bl_i[0], start_y), bl_i, (0,0,255), 2)
            # annotate measurements and deltas
            cv2.putText(
                debug,
                f"{col} M({results[col]['measured_mm'][0]},{results[col]['measured_mm'][1]}) " +
                f"D({results[col]['delta_mm'][0]},{results[col]['delta_mm'][1]})",
                (bl_i[0]+5, bl_i[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1
            )

        name = os.path.basename(path)
        print(name, json.dumps(results, indent=2))
        with open(f"./measurement/{name}.json", "w") as f:
            json.dump(results, f, indent=2)
        cv2.imwrite(f"./measurement/debug/dbg_{name}", debug)


extracted_left_3.png {
  "C": {
    "measured_mm": [
      2.329,
      2.178
    ],
    "delta_mm": [
      0.329,
      0.178
    ]
  },
  "M": {
    "measured_mm": [
      0.89,
      0.246
    ],
    "delta_mm": [
      -0.39,
      0.254
    ]
  },
  "Y": {
    "measured_mm": [
      1.908,
      0.342
    ],
    "delta_mm": [
      -0.092,
      0.158
    ]
  }
}
extracted_left_2.png {
  "C": {
    "measured_mm": [
      2.314,
      2.187
    ],
    "delta_mm": [
      0.314,
      0.187
    ]
  },
  "M": {
    "measured_mm": [
      0.88,
      0.254
    ],
    "delta_mm": [
      -0.38,
      0.246
    ]
  },
  "Y": {
    "measured_mm": [
      1.902,
      0.349
    ],
    "delta_mm": [
      -0.098,
      0.151
    ]
  }
}
extracted_left_1.png {
  "C": {
    "measured_mm": [
      2.333,
      2.174
    ],
    "delta_mm": [
      0.333,
      0.174
    ]
  },
  "M": {
    "measured_mm": [
      0.868,
      0.255
    ],
    "delta_mm": [
      -0.368,
      0.245
    ]
  },


In [33]:
import cv2
import numpy as np
import glob
import os
import json

# --- Utility functions from reference code ---

def order_points(pts: np.ndarray) -> np.ndarray:
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect = np.zeros((4,2), dtype="float32")
    rect[0] = pts[np.argmin(s)]      # top-left
    rect[2] = pts[np.argmax(s)]      # bottom-right
    rect[1] = pts[np.argmin(diff)]   # top-right
    rect[3] = pts[np.argmax(diff)]   # bottom-left
    return rect


def extract_marker(image: np.ndarray) -> np.ndarray | None:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, bin_ = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cnts, _ = cv2.findContours(255 - bin_, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cands = []
    for c in cnts:
        a = cv2.contourArea(c)
        if a < 5000:
            continue
        eps = 0.03 * cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, eps, True)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            pts = approx.reshape(4,2)
            w = np.linalg.norm(pts[0] - pts[1])
            h = np.linalg.norm(pts[1] - pts[2])
            if abs(1 - w/h) < 0.3:
                cands.append((a, pts))
    if not cands:
        return None
    _, best = max(cands, key=lambda x: x[0])
    rect = order_points(best)
    size = int(max(
        np.linalg.norm(rect[0]-rect[1]),
        np.linalg.norm(rect[1]-rect[2]),
        np.linalg.norm(rect[2]-rect[3]),
        np.linalg.norm(rect[3]-rect[0])
    ))
    dst = np.array([[0,0],[size-1,0],[size-1,size-1],[0,size-1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (size, size))


def detect_bottom_left(img: np.ndarray, hsv_range: tuple, min_area_ratio=0.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array(hsv_range[0]), np.array(hsv_range[1]))
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    areas = [cv2.contourArea(c) for c in cnts]
    mx = max(areas)
    big = [c for c,a in zip(cnts,areas) if a >= mx * min_area_ratio]
    for c in sorted(big, key=cv2.contourArea, reverse=True):
        hull = cv2.convexHull(c)
        eps = 0.02 * cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, eps, True).reshape(-1,2)
        if len(approx) == 4 and cv2.isContourConvex(approx):
            rect = order_points(approx)
            bottom_pts = sorted(rect, key=lambda p: p[1], reverse=True)[:2]
            bl = tuple(sorted(bottom_pts, key=lambda p: p[0])[0])
            return bl  # (x_px, y_px) in top-left origin
    return None


if __name__ == "__main__":
    # HSV color ranges (as in reference)
    HSV = {
        'C': ((90,80,80),(130,255,255)),
        'M': ((130,50,70),(170,255,255)),
        'Y': ((20,80,80),(40,255,255)),
        'K': ((0,0,0),(180,255,50)),  # approximate black range
    }

    # 목표점 비율 (x, y)
    T_RATIOS = {
        'K': (0.1, 0.6),
        'C': (0.6, 0.6),
        'M': (0.1, 0.1),
        'Y': (0.6, 0.1),
    }

    # 출력 디렉토리 생성
    os.makedirs('./measurement', exist_ok=True)
    os.makedirs('./measurement/debug', exist_ok=True)

    for path in glob.glob('./output/extracted_*.png'):
        orig = cv2.imread(path)
        cropped = extract_marker(orig)
        name = os.path.basename(path)
        if cropped is None:
            print(f"❌ Marker not found in {name}")
            continue

        # 이미지 크기 및 mm 변환 (전체 크기 = 5mm)
        h_px, w_px = cropped.shape[:2]
        mm_per_px_x = 5.0 / w_px
        mm_per_px_y = 5.0 / h_px

        debug = cropped.copy()
        results = {}

        # 각 색상별 P, T, Δ 계산
        for col, hsv_range in HSV.items():
            P = detect_bottom_left(cropped, hsv_range)
            if P is None:
                results[col] = None
                continue

            x_px, y_px = P
            # bottom-left 원점 기준 y
            y_bottom = h_px - y_px

            # 픽셀 → mm
            P_mm = (x_px * mm_per_px_x, y_bottom * mm_per_px_y)

            # 목표점 T 픽셀 및 mm
            tx_px = T_RATIOS[col][0] * w_px
            ty_px = T_RATIOS[col][1] * h_px
            T_mm = (tx_px * mm_per_px_x, (h_px - ty_px) * mm_per_px_y)

            # 이동량 Δ = T - P
            delta_mm = (T_mm[0] - P_mm[0], T_mm[1] - P_mm[1])

            results[col] = {
                'P_mm': (round(P_mm[0],3), round(P_mm[1],3)),
                'T_mm': (round(T_mm[0],3), round(T_mm[1],3)),
                'delta_mm': (round(delta_mm[0],3), round(delta_mm[1],3)),
            }

            # 디버그: P 위치에 초록 점, 좌표 및 Δ 표기
            bl_i = (int(x_px), int(y_px))
            cv2.circle(debug, bl_i, 6, (0,255,0), -1)
            cv2.putText(
                debug,
                f"{col}: P({results[col]['P_mm'][0]},{results[col]['P_mm'][1]})",
                (bl_i[0]+5, bl_i[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1
            )
            cv2.putText(
                debug,
                f"D={results[col]['delta_mm'][0]},{results[col]['delta_mm'][1]}",
                (bl_i[0]+5, bl_i[1]+15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1
            )

        # 결과 저장
        with open(f"./measurement/{name}.json", "w") as f:
            json.dump(results, f, indent=2)
        cv2.imwrite(f"./measurement/debug/dbg_{name}", debug)
        print(f"✅ Processed {name}")


✅ Processed extracted_left_3.png
✅ Processed extracted_left_2.png
✅ Processed extracted_left_1.png
✅ Processed extracted_left_4.png
✅ Processed extracted_20250718_145610755.png
✅ Processed extracted_20250718_145635599.png
✅ Processed extracted_20250718_145634047.png
✅ Processed extracted_20250718_145634895.png
✅ Processed extracted_right_1.png
✅ Processed extracted_right_3.png
✅ Processed extracted_right_2.png
✅ Processed extracted_right_4.png
